<a href="https://colab.research.google.com/github/linyuehzzz/traffic_data_aggregation/blob/master/data_agg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Mount on Google Drive**

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


**Add file path**

In [0]:
import os
os.chdir("/content/gdrive/My Drive/Colab Notebooks/traffic_data_aggregation")
!ls

data  data_agg.ipynb


**Prepare TomTom data**

In [0]:
import pandas as pd
import numpy as np
import json
import datetime
import os
import random
from urllib.request import urlopen, quote
from numbers import Number

os.chdir("/content/gdrive/My Drive/Colab Notebooks/traffic_data_aggregation")
%cd data

def get_response(url):
  req = urlopen(url)
  res = req.read().decode()
  temp = json.loads(res)
  return temp


def get_key(file):
  with open(file, "r") as f:
    return f.read()


def get_tomtom_all(file1, file2, key):
  with open(file2, "w") as f2:
    with open(file1, "r") as f1:
      all_lines = pd.read_csv(f1)
      random.shuffle(all_lines.values)
      for line in all_lines.values:
        lat = line[7]
        lon = line[8]
        url = "https://api.tomtom.com/traffic/services/4/flowSegmentData/absolute/10/json?point=" + \
        str(lat) + "%2C" + str(lon) + "&unit=KMPH&key=" + key
        # print(url)

        # print(get_response(url))
        spd = get_response(url)["flowSegmentData"]["currentSpeed"]
        spd_ff = get_response(url)["flowSegmentData"]["freeFlowSpeed"] # free flow speed
        tra_time = get_response(url)["flowSegmentData"]["currentTravelTime"]
        tra_time_ff = get_response(url)["flowSegmentData"]["freeFlowTravelTime"] # free flow travel time
        con = get_response(url)["flowSegmentData"]["confidence"]
        clo = get_response(url)["flowSegmentData"]["roadClosure"]
        
        str_temp = ""
        for i in line:
          if isinstance(i, Number):
            str_temp = str_temp + str(i) + ","
          else:
            str_temp = str_temp + i + ","
        str_temp = str(datetime.datetime.now()) + str_temp + str(spd) + "," + str(spd_ff) + "," + \
        str(tra_time) + "," + str(tra_time_ff) + "," + str(con) + "," + str(clo) + "\n"

        f2.write(str_temp)


file_key = "key.txt"
file_pts = "tomtom_spl_pts.csv"
file_output = "d04_tomtom_" + str(datetime.date.today()) + ".csv"
key = get_key(file_key)
get_tomtom_all(file_pts, file_output, key)


/content/gdrive/My Drive/Colab Notebooks/traffic_data_aggregation/data


CertificateError: ignored